In [12]:
import os
import cv2
import json
import numpy as np
from ultralytics import YOLO
from glob import glob

# ==== CONFIG ====
MODEL_PATH = r"C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\runs\train\jetbot_groundview\weights\epoch60.pt"
IMAGE_DIR = r"C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images"
OUT_DIR = os.path.join(IMAGE_DIR, "eval_debug")
os.makedirs(OUT_DIR, exist_ok=True)

# Model input size: this MUST match your model's training size!
MODEL_IMG_SIZE = 512  # or 640, whatever you trained with!

CLASS_NAMES = {0: "jetbot1", 1: "jetbot2", 2: "jetbot3", 3: "red_box"}

def draw_box(img, box, label, color, thickness=2):
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, max(y1 - 10, 10)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, lineType=cv2.LINE_AA)

def yolo_to_xyxy(cls_id, cx, cy, bw, bh, w, h):
    # Converts YOLO txt normalized coords to pixel coords
    x1 = int((cx - bw / 2) * w)
    y1 = int((cy - bh / 2) * h)
    x2 = int((cx + bw / 2) * w)
    y2 = int((cy + bh / 2) * h)
    return [x1, y1, x2, y2]

def resize_and_pad(img, target_size):
    h, w = img.shape[:2]
    scale = min(target_size / w, target_size / h)
    nw, nh = int(w * scale), int(h * scale)
    img_resized = cv2.resize(img, (nw, nh))
    top = (target_size - nh) // 2
    bottom = target_size - nh - top
    left = (target_size - nw) // 2
    right = target_size - nw - left
    img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(114, 114, 114))
    return img_padded, scale, left, top

image_paths = sorted([
    f for f in glob(os.path.join(IMAGE_DIR, "*.png"))
    if "debug" not in os.path.basename(f)
])[:100]

model = YOLO(MODEL_PATH)

for img_path in image_paths:
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # 1. Preprocess for model: resize and pad to model size
    img_input, scale, pad_x, pad_y = resize_and_pad(img, MODEL_IMG_SIZE)
    
    # 2. Run model on resized/padded input
    results = model(img_input, imgsz=MODEL_IMG_SIZE, verbose=False)[0]

    # 3. Map predicted boxes from model input size back to original image
    for box, cls, conf in zip(results.boxes.xyxy.cpu(), results.boxes.cls.cpu(), results.boxes.conf.cpu()):
        # Remove padding, then rescale
        x1, y1, x2, y2 = box
        # Remove padding
        x1 = (x1 - pad_x) / scale
        y1 = (y1 - pad_y) / scale
        x2 = (x2 - pad_x) / scale
        y2 = (y2 - pad_y) / scale
        # Clip to image boundaries
        x1 = int(max(0, min(x1, w - 1)))
        y1 = int(max(0, min(y1, h - 1)))
        x2 = int(max(0, min(x2, w - 1)))
        y2 = int(max(0, min(y2, h - 1)))
        label = f"{CLASS_NAMES.get(int(cls), str(int(cls)))} {conf:.2f}"
        draw_box(img, (x1, y1, x2, y2), label, (0, 255, 0))

    # 4. Draw GT boxes (BLUE) -- always correct
    txt_path = img_path.replace(".png", ".txt")
    if os.path.exists(txt_path):
        with open(txt_path, "r") as f:
            for line in f:
                if not line.strip():
                    continue
                cls_id, cx, cy, bw, bh = map(float, line.strip().split())
                xyxy = yolo_to_xyxy(cls_id, cx, cy, bw, bh, w, h)
                label = f"GT {CLASS_NAMES.get(int(cls_id), str(int(cls_id)))}"
                draw_box(img, xyxy, label, (255, 0, 0))

    # 5. Draw optional pose from JSON (ORANGE)
    json_path = img_path.replace(".png", ".json")
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            data = json.load(f)
            pose_objects = data.get("pose_data", []) if isinstance(data, dict) else data
            for idx, obj in enumerate(pose_objects):
                if "robot_id" in obj and "relative_position" in obj:
                    rel_pos = obj["relative_position"]
                    yaw = obj.get("yaw", 0)
                    label = f"{obj['robot_id']} pos=({rel_pos[0]:.2f}, {rel_pos[1]:.2f}) yaw={yaw:.2f}"
                    cv2.putText(img, label, (10, h - 10 - 20 * idx),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 1)

    out_path = os.path.join(OUT_DIR, os.path.basename(img_path))
    cv2.imwrite(out_path, img)

print(f"✅ Done. Annotated images saved to: {OUT_DIR}")


✅ Done. Annotated images saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\jetbot_groundview_dataset\val\images\eval_debug


In [9]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from glob import glob

# ==== CONFIG ====
MODEL_PATH = r"C:\Users\fredr\BTH\Robotik Project\Robotics\YOLOv8-training\runs\train\jetbot_groundview\weights\epoch280.pt"
IMAGE_DIR = r"C:\temp"
OUT_DIR = os.path.join(IMAGE_DIR, "debug_diags")
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_IMG_SIZE = 512  # Model input size
CLASS_NAMES = {0: "jetbot1", 1: "jetbot2", 2: "jetbot3", 3: "red_box"}

def draw_box(img, box, label, color, thickness=2):
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, max(y1 - 10, 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, lineType=cv2.LINE_AA)

def yolo_to_xyxy(cx, cy, bw, bh, w, h):
    x1 = int((cx - bw / 2) * w)
    y1 = int((cy - bh / 2) * h)
    x2 = int((cx + bw / 2) * w)
    y2 = int((cy + bh / 2) * h)
    return [x1, y1, x2, y2]

# Axis-swap/flip/scale diagnostic conversions
def all_gt_variants(cls_id, cx, cy, bw, bh, w, h):
    # Original
    orig = yolo_to_xyxy(cx, cy, bw, bh, w, h)
    # Swap x/y, bw/bh
    swap = yolo_to_xyxy(cy, cx, bh, bw, w, h)
    # Flip X (1-cx), keep others
    flipx = yolo_to_xyxy(1-cx, cy, bw, bh, w, h)
    # Flip Y (1-cy), keep others
    flipy = yolo_to_xyxy(cx, 1-cy, bw, bh, w, h)
    # Both flips
    flipxy = yolo_to_xyxy(1-cx, 1-cy, bw, bh, w, h)
    return {
        "GT_orig_BLUE": (orig, (255, 0, 0)),
        "GT_swap_RED": (swap, (0, 0, 255)),
        "GT_flipX_YELLOW": (flipx, (0, 255, 255)),
        "GT_flipY_GREEN": (flipy, (0, 255, 0)),
        "GT_flipXY_MAG": (flipxy, (255, 0, 255))
    }

# Prepare test images
image_paths = sorted([f for f in glob(os.path.join(IMAGE_DIR, "*.png")) if "debug" not in os.path.basename(f)])[:10]
model = YOLO(MODEL_PATH)

for img_path in image_paths:
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Model inference
    img_input = cv2.resize(img, (MODEL_IMG_SIZE, MODEL_IMG_SIZE))
    results = model(img_input, imgsz=MODEL_IMG_SIZE, verbose=False)[0]

    # Map prediction boxes back to original image
    scale = min(MODEL_IMG_SIZE/w, MODEL_IMG_SIZE/h)
    nw, nh = int(w * scale), int(h * scale)
    pad_x = (MODEL_IMG_SIZE - nw) // 2
    pad_y = (MODEL_IMG_SIZE - nh) // 2

    for box, cls, conf in zip(results.boxes.xyxy.cpu(), results.boxes.cls.cpu(), results.boxes.conf.cpu()):
        x1, y1, x2, y2 = box
        x1 = (x1 - pad_x) / scale
        y1 = (y1 - pad_y) / scale
        x2 = (x2 - pad_x) / scale
        y2 = (y2 - pad_y) / scale
        x1 = int(max(0, min(x1, w-1)))
        y1 = int(max(0, min(y1, h-1)))
        x2 = int(max(0, min(x2, w-1)))
        y2 = int(max(0, min(y2, h-1)))
        label = f"{CLASS_NAMES.get(int(cls), str(int(cls)))} {conf:.2f}"
        draw_box(img, (x1, y1, x2, y2), label, (0, 255, 0))  # GREEN for model prediction

    # Draw all GT variants
    txt_path = img_path.replace(".png", ".txt")
    if os.path.exists(txt_path):
        with open(txt_path, "r") as f:
            for line in f:
                if not line.strip(): continue
                parts = line.strip().split()
                if len(parts) < 5: continue
                cls_id, cx, cy, bw, bh = map(float, parts[:5])
                gt_boxes = all_gt_variants(cls_id, cx, cy, bw, bh, w, h)
                for key, (box, color) in gt_boxes.items():
                    draw_box(img, box, key, color)

    out_path = os.path.join(OUT_DIR, os.path.basename(img_path))
    cv2.imwrite(out_path, img)

print(f"✅ Diagnostics done! Check {OUT_DIR} for all GT variants and predictions.")


✅ Diagnostics done! Check C:\temp\debug_diags for all GT variants and predictions.
